In [ ]:
import pandas as pd
import subprocess
import sys
import matplotlib.pyplot as plt
from matplotlib import cm 
import numpy as np

In [ ]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [ ]:
ref_dir_path = f'/data/LNG/vitaled2/1kgenomes'
ashk_data = f'/data/LNG/iwakih2/dataset/Jew/GSE23636'
onekg_demog_path = f'{ref_dir_path}/igsr_samples.tsv'
out_ancestry_file = f'{ref_dir_path}/ref_panel_ancestry.txt'
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel'

In [ ]:
# first, create list of Ashkenazis by FID and IID
ashk = pd.read_csv(f'{ashk_data}.fam', header=None, sep=' ')

# add ashkenazis with labels to df
ancestry = pd.DataFrame()
ancestry[['FID','IID']], ancestry['label'] = ashk[[0,1]], 'AJ'

In [ ]:
# read 1kG and label 
onekg_demog = pd.read_csv(onekg_demog_path, sep='\t')
onekg_demog['FID'], onekg_demog['IID'] = onekg_demog['Sample name'], onekg_demog['Sample name']
onekg_demog['label'] = onekg_demog['Superpopulation code']

# create separate label for African American (ASW) and Caribbean (ACB) label to AAC to match Neuro+
onekg_demog.loc[(onekg_demog['Population code'] == 'ASW') | (onekg_demog['Population code'] == 'ACB'), 'label'] = 'AAC'

# create separate label for Finnish (FIN) to match Neuro+
onekg_demog.loc[onekg_demog['Population code'] == 'FIN', 'label'] = 'FIN'
onekg_demog.label.unique()

In [ ]:
# check if all populations in GP2 are present.... Add Ashkenazi (AJ) and EXCLUDE FINNISH!!!!
#figure out the situation with CHD
pop_dict = {
    'AMR': ['MXL','CLM','PEL','PUR'],
    'EAS': ['JPT','CDX','CHB','CHS','KHV','CHD'],
    'EUR': ['TSI','IBS','GBR','CEU', 'AJ', 'FIN'],
    'SAS': ['PJL','ITU','STU','GIH','BEB'],
    'AFR': ['GWD','MSL','ESN','GWJ','YRI','LWK','GWF','GWW'],
    'AAC': ['ASW','ACB']
}

pop_list = [pop for poplist in [pop for supergroup, pop in pop_dict.items()] for pop in poplist]

print('Counts per population code in 1kG')
total = 0
for pop in pop_list:
    count = (onekg_demog['Population code'] == pop).sum()
    print(pop, count)
    total += count
    
print(f'Total across population codes in 1kG: {total}')
print()
# keep only onekg_demog populations that are in population list (pops in neuro+ chip, given by GP2)
onekg_demog = onekg_demog.loc[onekg_demog['Population code'].isin(pop_list)]

#append onekg_demog to ancestry
ancestry_final = ancestry.append(onekg_demog[['FID','IID','label']], ignore_index=True)
ancestry_final.to_csv(out_ancestry_file, header=True, index=False, sep='\t')
ancestry_final[['FID','IID']].to_csv(f'{ref_dir_path}/gp2_keep.txt', sep='\t', header=None, index=None)
print()
print('Counts per superpopulation code in 1kG + Ashkenazi')
print(ancestry_final.label.value_counts(dropna=False))
print(f'Total across superpopulation codes in 1kG + Ashkenazi: {ancestry_final.label.value_counts(dropna=False).sum()}')

In [ ]:
# prune the shit out of the ref panel to prep for ancestry test
# keep only country codes found in GP2 neuro+ chip (excluding Finnish and including Ashkenazis)
ref_panel_prune1 = f'{ref_panel}_maf_geno_hwe'
ref_panel_final_prune = f'{ref_panel}_gp2_pruned'

plink_cmd1 = f'plink --bfile {ref_panel}\
 --maf 0.05\
 --geno 0.01\
 --hwe 0.0001\
 --autosome\
 --keep {ref_dir_path}/gp2_keep.txt\
 --exclude {ref_dir_path}/hg38_exclusion_regions.txt\
 --make-bed\
 --out {ref_panel_prune1}' 

plink_cmd2 = f'plink --bfile {ref_panel_prune1}\
 --indep-pairwise 1000 10 0.02\
 --autosome\
 --out {ref_dir_path}/pruned_data'

plink_cmd3 = f'plink --bfile {ref_panel_prune1}\
 --extract {ref_dir_path}/pruned_data.prune.in\
 --make-bed\
 --out {ref_panel_final_prune}'

cmds = [plink_cmd1, plink_cmd2, plink_cmd3]

# for cmd in cmds:
#     shell_do(cmd)

In [ ]:
# now, merge ancestry with pruned ref panel to see how many in each ancestry group
pruned_ref_fam = pd.read_csv(f'{ref_panel_final_prune}.fam', sep=' ', header=None)
total_pruned_ref_df = pruned_ref_fam.merge(ancestry_final, how='left', left_on=[0,1], right_on=['FID','IID'])

total_pruned_ref_df.label.value_counts()

# FastStructure test with Ref Panel

In [ ]:
# Now, test with faststructure
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned'
ancestry_labels = f'{ref_dir_path}/ref_panel_ancestry.txt'
structure_out = '/data/vitaled2/ref_panel/temp/1kg_ashkj_ref_panel_gp2_structure'
fam = pd.read_csv(f'{ref_panel}.fam', sep=' ', header=None)
structure = f'/data/vitaled2/ref_panel/fastStructure/structure.py'

In [ ]:
structure_run = f'python {structure}\
 -K 8\
 --input={ref_panel}\
 --output={structure_out}'

# # need to run in terminal in conda virtualenv with python2
print(structure_run)

In [ ]:
# after fastStructure finishes running:
# read Q file and create labels
q_df = pd.read_csv(f'{structure_out}.8.meanQ', header=None, sep='\s+')
q_df.columns = [f'pop{i}' for i in range(1,9)]
fam['highest_2_pops'] = q_df.apply(lambda s: s.abs().nlargest(2).index.tolist(), axis=1)
fam['highest_pop'] = q_df.idxmax(axis=1)

q_df['FID'], q_df['IID'], q_df['highest_pop'], q_df['highest_2_pops'] = fam[0], fam[1], fam['highest_pop'], fam['highest_2_pops']


In [ ]:
pop = pd.read_csv(ancestry_labels, sep='\t')
q_pop_merged = q_df.merge(pop, left_on=['FID','IID'], right_on=['FID','IID'])
print(q_pop_merged['highest_pop'].value_counts())
# q_pop_merged['label'].value_counts()
q_pop_merged.to_csv(f'{ref_dir_path}/pca_labeled_faststructure.txt', sep='\t', header=True, index=False)


In [ ]:
q_pop_merged.describe()
q_pop_merged['label'].unique()

In [ ]:
q_pop_merged[q_pop_merged['label'] == 'AJ'].describe() # almost 100% pop7
q_pop_merged[q_pop_merged['label'] == 'EUR'].describe() # almost 100% pop5, next highest mean is pop8 which makes sense
q_pop_merged[q_pop_merged['label'] == 'FIN'].describe() # almost 100% pop5 ... FIN is questionable as not really unique, how does this work with PCA?
q_pop_merged[q_pop_merged['label'] == 'EAS'].describe() # almost 93% pop3
q_pop_merged[q_pop_merged['label'] == 'AMR'].describe() # 53% POP5 and 37% POP6
q_pop_merged[q_pop_merged['label'] == 'SAS'].describe() # almost 95% POP8
q_pop_merged[q_pop_merged['label'] == 'AAC'].describe() # almost 90% POP1
q_pop_merged[q_pop_merged['label'] == 'AFR'].describe() # almost 100% POP1

In [ ]:
#pop2 only contains 1 sample
high_pop2_df = q_pop_merged[q_pop_merged['pop2'] > 0.8]
high_pop2_df['label'].value_counts()

In [ ]:
#remove samples
high_pop2_df[['FID','IID']].to_csv('/data/vitaled2/ref_panel/rm_samples.txt', header=False, index=False, sep='\t')

In [ ]:
# now remove samples with plink

plink_rm_cmd = f'plink --bfile {ref_panel}\
 --maf 0.05\
 --geno 0.01\
 --hwe 0.0001\
 --autosome\
 --remove /data/vitaled2/ref_panel/rm_samples.txt\
 --make-bed\
 --out {ref_panel}_final' 

# shell_do(plink_rm_cmd)


In [ ]:
# now, run faststructure with removed rogue sample
structure_run2 = f'python {structure}\
 -K 8\
 --input={ref_panel}_final\
 --output={ref_panel}_final_structure'

# need to run in terminal in conda virtualenv with python2
print(structure_run2)

In [ ]:
# now, take a look at faststructure after removing rogue sample
q_df = pd.read_csv(f'{ref_panel}_final_structure.8.meanQ', header=None, sep='\s+')
q_df.columns = [f'pop{i}' for i in range(1,9)]
fam['highest_2_pops'] = q_df.apply(lambda s: s.abs().nlargest(2).index.tolist(), axis=1)
fam['highest_pop'] = q_df.idxmax(axis=1)

q_df['FID'], q_df['IID'], q_df['highest_pop'], q_df['highest_2_pops'] = fam[0], fam[1], fam['highest_pop'], fam['highest_2_pops']


In [ ]:
pop = pd.read_csv(ancestry_labels, sep='\t')
q_pop_merged = q_df.merge(pop, left_on=['FID','IID'], right_on=['FID','IID'])
print(q_pop_merged['highest_pop'].value_counts())
# q_pop_merged['label'].value_counts()
q_pop_merged.to_csv(f'{ref_dir_path}/labeled_faststructure.txt', sep='\t', header=True, index=False)
q_pop_merged['label'].to_csv(f'{ref_dir_path}/faststructure_labels.txt', sep='\t', header=False, index=False)

In [ ]:
q_pop_merged.describe()
# q_pop_merged['label'].unique()

In [ ]:
q_pop_merged[q_pop_merged['label'] == 'AJ'].describe() # almost 100% pop4
# q_pop_merged[q_pop_merged['label'] == 'EUR'].describe() # almost 100% pop8, next highest mean is pop4 which makes sense
# q_pop_merged[q_pop_merged['label'] == 'FIN'].describe() # almost 100% pop8
# q_pop_merged[q_pop_merged['label'] == 'EAS'].describe() # almost 89% pop6, 10% POP5
# q_pop_merged[q_pop_merged['label'] == 'AMR'].describe() # 50% POP8 and 40% POP2
# q_pop_merged[q_pop_merged['label'] == 'SAS'].describe() # almost 91% POP3
# q_pop_merged[q_pop_merged['label'] == 'AAC'].describe() # 82% POP7 13% POP8
# q_pop_merged[q_pop_merged['label'] == 'AFR'].describe() # almost 91% POP7

In [ ]:
#distruct run for visualization
distruct_run = f'python /data/vitaled2/ref_panel/fastStructure/distruct.py\
 -K 8\
 --input={ref_panel}_final_structure\
 --output={ref_panel}_distruct\
 --popfile={ref_dir_path}/faststructure_labels.txt\
 --title="GP2 Reference Panel Ancestries"'

print(distruct_run)

# Test faststructure with spanish GWAS data + ref panel

# Test PCA with spanish GWAS data + ref panel

In [ ]:
# steps for detecting ancestry outliers used on spanish gwas data
geno_path = '/data/vitaled2/test_data/spanish_gwas2/SPAIN2ndpart_pheno_call_rate_het_sex_heterozyg_hapmap_relatedness_variant'
out_path = '/data/vitaled2/test_data/spanish_gwas2/'
ref_path = '/data/LNG/vitaled2/1kgenomes/1kg_ashkj_ref_panel_final'

bash1 = f"plink --bfile {geno_path} --bmerge {ref_path} --out {out_path}bin_snplis --make-bed"
bash2 = f"plink --bfile {geno_path} --flip {out_path}bin_snplis-merge.missnp --make-bed --out {geno_path}_flip"
bash3 = f"plink --bfile {geno_path}_flip --bmerge {ref_path} --out {out_path}bin_snplis --make-bed"
bash4 = f"plink --bfile {geno_path}_flip --exclude {out_path}bin_snplis-merge.missnp --out {geno_path}_flip_pruned --make-bed"
bash5 = f"plink --bfile {geno_path}_flip_pruned --bmerge {ref_path} --out {out_path}bin_snplis --make-bed"
bash6 = f"plink --bfile {out_path}bin_snplis --geno 0.01 --out {out_path}pca --make-bed --pca 8"


cmds = [bash1, bash2, bash3, bash4, bash5, bash6]

# for cmd in cmds:
#     shell_do(cmd)


In [ ]:
pca_path = '/data/vitaled2/test_data/spanish_gwas2/pca.eigenvec'

pca = pd.read_csv(pca_path, sep=' ', header=None)
pca_columns = ['FID','IID'] + [f"pc{i-1}" for i in range(2,len(pca.columns))]
pca.columns = pca_columns


In [ ]:
new_samples_path = f'{geno_path}.fam'
new_samples_fam = pd.read_csv(new_samples_path, sep=' ', header=None)
labeled_samples = new_samples_fam.loc[:,[0,1]]
labeled_samples.loc[:,'label'] = 'new'
labeled_samples.rename(columns={0:'FID',1:'IID'}, inplace=True)


In [ ]:
# combine labeled samples with labeled ref panel ids and merge with pca
combined_labels = labeled_samples.append(total_pruned_ref_df.loc[:, ['FID','IID','label']])
# combined_labels
labeled_pca = pca.merge(combined_labels, how='left', on=['FID','IID'])
print(labeled_pca.label.value_counts())


In [ ]:
# now plot PCs
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
cmap = cm.get_cmap('tab10')
targets = list(labeled_pca.label.unique())
targets = ['new', 'EUR']
colors = cmap(np.linspace(0, 1, len(targets)))
for target, color in zip(targets,colors):
    indicesToKeep = labeled_pca['label'] == target
    ax.scatter(labeled_pca.loc[indicesToKeep, 'pc1']
               , labeled_pca.loc[indicesToKeep, 'pc2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
{target:color for target,color in zip(targets,colors)}

In [ ]:
pop_list=['AJ', 'EUR', 'FIN', 'EAS', 'AMR', 'SAS', 'AAC', 'AFR']
total_pop_list = pop_list.copy()
total_pop_list.insert(0, 'new')
print(pop_list)
print(total_pop_list)

In [ ]:
# def detect_pca_outliers(pca_df, pop_list=['AJ', 'EUR', 'FIN', 'EAS', 'AMR', 'SAS', 'AAC', 'AFR'], new_sample_label='new'):
#     total_pop_list = pop_list.copy()
#     total_pop_list.insert(0, new_sample_label)
    
#     cmap = cm.get_cmap('tab10')
#     colors = cmap(np.linspace(0, 1, len(total_pop_list)))
#     colormap = {pop:color for pop,color in zip(total_pop_list,colors)}
    
#     for pop in pop_list:
#         lowc1 = (pca_df.loc[(pca_df.label == pop), 'pc1'].mean()) - (6 * pca_df.loc[(pca_df.label == pop), 'pc1'].std())
#         highc1 = (pca_df.loc[(pca_df.label == pop), 'pc1'].mean()) + (6 * pca_df.loc[(pca_df.label == pop), 'pc1'].std())
#         lowc2 = (pca_df.loc[(pca_df.label == pop), 'pc2'].mean()) - (6 * pca_df.loc[(pca_df.label == pop), 'pc2'].std())
#         highc2 = (pca_df.loc[(pca_df.label == pop), 'pc2'].mean()) + (6 * pca_df.loc[(pca_df.label == pop), 'pc2'].std())
#         pruned_pca = pca_df.loc[(pca_df.pc1 >= lowc1) & (pca_df.pc1 <= highc1) & (pca_df.pc2 >= lowc2) & (pca_df.pc2 <= highc2)]
        
#         fig = plt.figure(figsize = (8,8))
#         ax = fig.add_subplot(1,1,1) 
#         ax.set_xlabel('Principal Component 1', fontsize = 15)
#         ax.set_ylabel('Principal Component 2', fontsize = 15)
#         ax.set_title('2 component PCA', fontsize = 20)

#         targets = [new_sample_label, pop]

#         for target in targets:
#             indicesToKeep = pruned_pca['label'] == target
#             ax.scatter(pruned_pca.loc[indicesToKeep, 'pc1']
#                        , pruned_pca.loc[indicesToKeep, 'pc2']
#                        , c = colormap[target]
#                        , s = 50)
#         ax.legend(targets)
#         ax.grid()


        
# detect_pca_outliers(labeled_pca)       

###### 

In [ ]:
# pruned_pca = labeled_pca.loc[(labeled_pca.pc1 >= lowc1) & (labeled_pca.pc1 <= highc1) & (labeled_pca.pc2 >= lowc2) & (labeled_pca.pc2 <= highc2)]

# fig = plt.figure(figsize = (8,8))
# ax = fig.add_subplot(1,1,1) 
# ax.set_xlabel('Principal Component 1', fontsize = 15)
# ax.set_ylabel('Principal Component 2', fontsize = 15)
# ax.set_title('2 component PCA', fontsize = 20)
# cmap = cm.get_cmap('tab10')
# targets = list(pruned_pca.label.unique())
# # targets = ['new', 'EUR', 'AJ']
# colors = cmap(np.linspace(0, 1, len(targets)))
# for target, color in zip(targets,colors):
#     indicesToKeep = pruned_pca['label'] == target
#     ax.scatter(pruned_pca.loc[indicesToKeep, 'pc1']
#                , pruned_pca.loc[indicesToKeep, 'pc2']
#                , c = color
#                , s = 50)
# ax.legend(targets)
# ax.grid()